In [1]:
# install arch from Github
#!pip install git+https://github.com/bashtage/arch.git

In [2]:
#!pip install git+git://github.com/pynbody/pynbody.git

In [3]:
#!pip install --upgrade numpy

In [4]:
#!pip install arch

In [5]:
#Compare 1000 coin tosses to a normal distribution
import scipy.stats as spst
import scipy
import numpy as np
import pandas as pd
import qrbook_funcs as qf
from scipy import stats
from tabulate import tabulate
from arch import arch_model
import math
import statistics 
import matplotlib.pyplot as plt
%matplotlib inline

### Work with training dataset 2015 to 2017

In [6]:
#Get 3 currencies until the end of #previous year. Form sample covariance matrix and do simple efficient frontier calculations

#Swiss franc, pound sterling, Japanese Yen
seriesnames=['DEXSZUS','DEXUSUK','DEXJPUS']
cdates_train, ratematrix_train = qf.GetFREDMatrix(seriesnames, startdate='2014-12-31', enddate='2017-12-29')

nobs, t = len(cdates_train), 0
while t < nobs:
    if all(np.isnan(ratematrix_train[t])):
        del ratematrix_train[t]
        del cdates_train[t]
        nobs -= 1
    else:
        t += 1

In [7]:
train=pd.DataFrame(ratematrix_train, index=cdates_train, columns=seriesnames)

In [8]:
train.head()

,DEXSZUS,DEXUSUK,DEXJPUS
2014-12-31,0.9934,1.5578,119.85
2015-01-02,1.0004,1.5361,120.20
2015-01-05,1.0079,1.5234,119.64
2015-01-06,1.0062,1.5174,118.26
2015-01-07,1.0160,1.5073,119.52


In [9]:
train_pound=pd.DataFrame(train['DEXUSUK'], index=cdates_train)

In [10]:
# check head and tail
# train_pound.head()
# train_pound.tail()

In [11]:
# compute log for every record
train_log=train_pound.apply(lambda x : np.log(x))

# compute first difference
train_final=train_log.diff()

# drop NA
train_final.dropna(inplace=True)

In [12]:
# check log return
# train_final.head()
# train_final.tail()

In [13]:
log_return=np.array(train_final)*1000

In [14]:
# Fit GARCH model
garch11=arch_model(log_return,p=1,q=1)
res=garch11.fit()

Iteration:      1,   Func. Count:      6,   Neg. LLF: 2438.2322919421513
Iteration:      2,   Func. Count:     14,   Neg. LLF: 2436.1124752121086
Iteration:      3,   Func. Count:     21,   Neg. LLF: 2435.7938703950276
Iteration:      4,   Func. Count:     28,   Neg. LLF: 2435.4938201888203
Iteration:      5,   Func. Count:     36,   Neg. LLF: 2435.445809423254
Iteration:      6,   Func. Count:     44,   Neg. LLF: 2435.416607205715
Iteration:      7,   Func. Count:     50,   Neg. LLF: 2435.361200733251
Iteration:      8,   Func. Count:     56,   Neg. LLF: 2435.2545754843422
Iteration:      9,   Func. Count:     62,   Neg. LLF: 2435.249841038235
Iteration:     10,   Func. Count:     68,   Neg. LLF: 2435.2497071544003
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2435.2497062670373
            Iterations: 10
            Function evaluations: 69
            Gradient evaluations: 10


In [15]:
print(res.summary())

                     Constant Mean - GARCH Model Results                      
Dep. Variable:                      y   R-squared:                      -0.000
Mean Model:             Constant Mean   Adj. R-squared:                 -0.000
Vol Model:                      GARCH   Log-Likelihood:               -2435.25
Distribution:                  Normal   AIC:                           4878.50
Method:            Maximum Likelihood   BIC:                           4896.99
                                        No. Observations:                  751
Date:                Sun, Nov 24 2019   Df Residuals:                      747
Time:                        22:22:14   Df Model:                            4
                               Mean Model                               
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
mu            -0.1348      0.240     -0.562      0.574 [ -0.605,  0.33

In [16]:
# #fit GARCH(1,1) model
# initparams=[.12,.85,.6]
# a,b,c=qf.Garch11Fit(initparams,log_return)

# #Display results
# print("a=%.3f" % a)
# print("b=%.3f" % b)
# print("c=%.3f" % c)

In [17]:
res.params

mu         -0.134769
omega       2.895833
alpha[1]    0.128741
beta[1]     0.812718
Name: params, dtype: float64

In [18]:
c=res.params[1]
a=res.params[2]
b=res.params[3]
print( "Stationary check (less than 1): ", a+b)

long_term_std=np.sqrt(12*c/(1-a-b))
print("Long term standard deviation is: ", long_term_std)

Stationary check (less than 1):  0.9414593760942198
Long term standard deviation is:  24.364007180919803


### Working with holdout dataset 2018 249 records

In [19]:
# Get the holdout dataset starting from 2017-12-29
cdates_holdout, ratematrix_holdout = qf.GetFREDMatrix(seriesnames, startdate='2017-12-29', enddate='2018-12-31')

nobs, t = len(cdates_holdout), 0
while t < nobs:
    if all(np.isnan(ratematrix_holdout[t])):
        del ratematrix_holdout[t]
        del cdates_holdout[t]
        nobs -= 1
    else:
        t += 1

In [20]:
# store holdout 2018 into dataframe
holdout=pd.DataFrame(ratematrix_holdout, index=cdates_holdout, columns=seriesnames)

In [21]:
# check length=250
# len(holdout['DEXUSUK'])

In [22]:
# holdout 2018 dataframe Pound ONLY
holdout_pound=pd.DataFrame(holdout['DEXUSUK'], index=cdates_holdout)

# compute log for every record
holdout_log=holdout_pound.apply(lambda x : np.log(x))

# compute first difference
holdout_final=holdout_log.diff()

# drop NA
holdout_final.dropna(inplace=True)

# check length
len(holdout_final)

# convert the series to a list as holdout
holdout_list=holdout_final['DEXUSUK'].tolist()

In [31]:
# number of days in 2018
t=249

# 1. create a stdgarch to store the estimated std series for 2018 
stdgarch=np.zeros(t)
# std of train dataset as the starting point for 2018 var series
stdgarch[0]=holdout_list[0]

# 2. series to hold de-garched series y[t]/sigma[t]
degarched=np.zeros(t)   
degarched[0]=holdout_list[0]/stdgarch[0]

In [24]:
# Compute GARCH(1,1) stddev's for 2018 using estimated parameters from training dataset 2015-2017
for i in range(1,t):
    #Note offset - i-1 observation of data is used for i estimate of std deviation
    previous=stdgarch[i-1]**2
    var=c+b*previous+a*(holdout_list[i-1])**2
    stdgarch[i]=np.sqrt(var)
    degarched[i]=(holdout_list[i])/stdgarch[i]

In [25]:
np.var(degarched)

0.002148868453534065

In [26]:
spst.kurtosis(degarched)

243.63777759358322

In [27]:
np.var(stdgarch)

0.10639818313513613

In [28]:
spst.kurtosis(stdgarch)

87.83733640832375

In [29]:
stdgarch[0]

0.006743091682213035

In [30]:
holdout_list[0]

0.004940102212646347